### Importing the libraries and Configuring the API Key

In [1]:
import pandas as pd
import numpy as np
import google.generativeai as genai
from dotenv import dotenv_values

In [2]:
df = pd.read_csv('dataset/tmdb_5000_movies_embed.csv')

In [3]:
df.head(5)

,title,overview,Embeddings
0,Avatar,"In the 22nd century, a paraplegic Marine is di...","[0.031759717, 0.0044186483, -0.025144888, -0.0..."
1,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...","[0.06617279, 0.008563878, -0.04165544, -0.0447..."
2,Spectre,A cryptic message from Bond’s past sends him o...,"[-0.012871856, -0.005491358, 0.012047383, -0.0..."
3,The Dark Knight Rises,Following the death of District Attorney Harve...,"[0.04840929, 0.03056959, -0.030966783, -0.0182..."
4,John Carter,"John Carter is a war-weary, former military ca...","[0.061523866, -0.032111533, -0.035515673, 0.00..."


In [4]:
df['Embeddings'] = df['Embeddings'].apply(lambda x: eval(x))

In [5]:
print(type(df['Embeddings'][0]))

<class 'list'>


In [6]:
ENV = dotenv_values(".env")
api_key = ENV['API_KEY']
genai.configure(api_key=api_key)

### Creating the Embedding function to the query

In [7]:
embedding_model = "models/embedding-001"

In [8]:
def embed_content(title, query, model, task_type):
    if query is None:
        return genai.embed_content(
            model=model,
            content=query,
            task_type=task_type,
        )["embedding"]
    else:
        return genai.embed_content(
            model=model,
            title=title,
            content=query,
            task_type=task_type,
        )["embedding"]

In [9]:
def get_suggestion(query, df, model):
    query_embedding = embed_content(
        None,
        query,
        model,
        "RETRIEVAL_QUERY",
    )
    
    # df['Embeddings'] = df['Embeddings'].apply(lambda x: np.array(x))
    scalars = np.dot(np.stack(df["Embeddings"]), query_embedding, out=None)
    index = np.argmax(scalars)
    return df.iloc[index][["title", "overview"]]

### Query of the user

In [10]:
query = "A movie about white rapper"
suggestion = get_suggestion(query, df, embedding_model)
print(f"Title: {suggestion.title}")
print(f"Description: {suggestion.overview}")

Title: 8 Mile
Description: The setting is Detroit in 1995. The city is divided by 8 Mile, a road that splits the town in half along racial lines. A young white rapper, Jimmy "B-Rabbit" Smith Jr. summons strength within himself to cross over these arbitrary boundaries to fulfill his dream of success in hip hop. With his pal Future and the three one third in place, all he has to do is not choke.


### Configuring the response from Gemini

In [11]:
generation_config = {
  "temperature": 0.5,
  "candidate_count": 1,
}

In [12]:
safety_settings = {
    'HARASSMENT': 'BLOCK_MEDIUM_AND_ABOVE',
    'HATE': 'BLOCK_MEDIUM_AND_ABOVE',
    'SEXUAL': 'BLOCK_MEDIUM_AND_ABOVE',
    'DANGEROUS': 'BLOCK_MEDIUM_AND_ABOVE',
}

In [13]:
prompt = f"Com base na [QUERY] sobre características/atributos de filmes: {query}. Elabore um breve resumo do filme: {suggestion.title}, considerando as características citadas na [QUERY]. Seja simpático e divertido, e busco me convencer a assistir a esse filme"

In [14]:
model = genai.GenerativeModel(
    model_name='gemini-1.0-pro',
    generation_config=generation_config, 
    safety_settings=safety_settings
)

In [15]:
response = model.generate_content(prompt)

### Response

In [16]:
print(response.text)

Prepare-se para uma viagem eletrizante ao mundo do rap em 8 Mile! Este filme icônico apresenta Eminem como Jimmy "B-Rabbit" Smith Jr., um aspirante a rapper branco que enfrenta desafios implacáveis em sua jornada para conquistar o respeito no cenário do hip-hop.

Com rimas afiadas como navalhas e batidas arrepiantes, 8 Mile captura a essência da cultura do rap e o poder transformador da música. Testemunhe as lutas pessoais de Jimmy, sua determinação inabalável e sua busca por autenticidade em um mundo que muitas vezes o julga por sua cor de pele.

Prepare-se para ser levado por uma história emocionante de superação, perseverança e o poder da autoexpressão. 8 Mile é um filme que vai te fazer bater a cabeça, te inspirar e te lembrar que mesmo as vozes mais improváveis podem fazer barulho.
